# teps 단어장 크롤링
- https://consulteps.com/myclass/page_group?pg_idx=11&utm_source=free&utm_medium=online_voca&utm_campaign=home_link

In [ ]:
# pip install pyperclip

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pyperclip
from tqdm import tqdm

In [ ]:
# 네이버 아이디, 패스워드 입력
id_value=input("네이버 id를 입력해주세요: ")
pw_value=input("네이버 pw를 입력해주세요: ")

# 셀레니움 시작
driver=webdriver.Chrome()
url="https://consulteps.com/myclass/page_group?pg_idx=11&utm_source=free&utm_medium=online_voca&utm_campaign=home_link"
driver.get(url)

# 네이버 클릭
driver.find_element(By.XPATH,'/html/body/div[1]/div/ul/li[1]/a').click()
driver.switch_to.window(driver.window_handles[1])

# 자신의 id, pw 입력
id_value=id_value
pw_value=pw_value

# id, pw값 전달, 입력
id_box = driver.find_element(By.ID, value="id")
id_box.click()
pyperclip.copy(id_value)
id_box.send_keys(Keys.CONTROL, 'v')
time.sleep(0.5)
pw_box = driver.find_element(By.ID, value="pw")
pw_box.click()
pyperclip.copy(pw_value)
pw_box.send_keys(Keys.CONTROL, 'v')
time.sleep(0.5)

driver.find_element(By.XPATH,'//*[@id="log.login"]').click()
time.sleep(0.5)
driver.switch_to.window(driver.window_handles[1])

# 새로운 기기 브라우저에서 로그인 -> 등록안함
driver.find_element(By.XPATH,'//*[@id="new.dontsave"]').click()
driver.switch_to.window(driver.window_handles[0])

time.sleep(0.5)

# html가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
pages=soup.select('.list>li')
pages=pages[:-4] #순위 아닌 데이터 제거, 2023년 3월 1일 기준 58개

root_url="https://consulteps.com/"
url_list=[]
for i in range(0, len(pages)):
    title=pages[i].select('a')[0].text
    url=pages[i].select('a')[0]['href']
    url=root_url+url
    url_list.append([title, url])
url_df=pd.DataFrame(url_list)
url_df.columns=['rank','url']

# 한 페이지씩 접속
url_list=list(url_df['url'])
rank_list=list(url_df['rank'])
word_df=pd.DataFrame()
for a in tqdm(range(len(url_list))):
    url=url_list[a]
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    words=soup.select('li.tit')
    word_list=[]
    for i in range(len(words)):
        eng=words[i].select('span')[0].text
        kor=words[i].select('span')[1].text
        word_list.append([eng, kor])
    word_tmp=pd.DataFrame(word_list)
    word_tmp['rank']=rank_list[a] #몇 순위 인지 표시
    word_df=pd.concat([word_df, word_tmp])
    time.sleep(1)
    
# 컬럼명 수정 및 인덱스 리셋
word_df.columns=['eng','kor','rank']
word_df.reset_index(inplace=True, drop=True)
word_df['rank']=word_df['rank'].apply(lambda x:str(x).split(' ')[2])
word_df=word_df[['rank','eng','kor']]

In [ ]:
word_df.to_excel('teps단어장.xlsx') #인덱스 포함